# PlotXspec example

## Basic PyXspec usage & additional plotting

This notebook is aimed at illustrating the _basic_ usage of PyXspec, as well as a few supporting methods related to plotting. The supporting methods are part of the [PlotXspec class available for download here](https://github.com/JohannesBuchner/BXA/tree/master/examples/xspec/bayesian-workflow). The notebook will cover how to load spectra, define a model, fit it to the data, and plot the results. PyXspec makes all Xspec functionality accessible through Python, so has a much broader range of options than covered here. For further details on PyXspec usage, please see the [documentation](https://heasarc.gsfc.nasa.gov/xanadu/xspec/python/html/index.html).

In the following, we will assume a basic familiarity with both Python and Xspec usage. Users will need to have a working HEASOFT installation, as well as a working Python3 installation. The examples focus on the 'out-of-the-box' usage of the PlotXSpec public methods. Some users may find it more helpful to use these methods as a template for their own plotting functions; the PlotXspec class includes several private methods that could also be of use as a guide (such as the ```_get_xspec_data()``` method).

_Please note_: this notebook relates to PyXspec only. For examples of using BXA and the PlotBXA class, please see the notebook [_tutorial_usage_plotbxa_](https://johannesbuchner.github.io/BXA/tutorial_usage_plotbxa.html). 

#### The notebook is structured as follows:
1) Basic loading and plotting data using PyXspec
2) Further plotting examples
3) A slightly more complicated example (multiple spectra & simultaneous fitting)

---
This notebook was tested with:
- HEASOFT 6.33.1
- Python 3.11.4

## 1) Loading and plotting data using PyXspec

We will start with a simple example: loading a single spectrum into Xspec and fitting a model to it. The example data are a NICER spectrum for a bright extragalactic nuclear ignition and the associated background spectrum. The data have been rebinned to a minimum bin count of 20, which means we can use $\chi^2$ statistics.

In [ ]:
# Loading the required modules
import xspec
import os,sys

# load the PlotXspec class and create an instance
sys.path.append(os.getcwd()+'/..')
from plot_xspec import PlotXspec
px = PlotXspec()

### A quick aside on getting more information on class methods in Python

If at any point you are unclear about what a method is supposed to do, or what arguments should be passed to it, you can make use of Python's built-in help functionality.

This will list all public methods included in the class:
```python
help(px)
```

And this will show the 'docstring' for a particular method:
```python
help(px.plot_model_and_data)
```

**The public methods in PlotXspec all have docstrings: these explain in more detail what the methods are supposed to and what additional arguments can be passed to make the best use of them.**

### A) Loading

In [ ]:
# Global settings for XSpec
xspec.Xset.abund = 'wilm' # Wilms et al. '00'
xspec.Xset.xsect = 'vern' # Verner et al. '96'
xspec.Xset.cosmo = '70 0 0.73' # Flat LambdaCDM

In [ ]:
# Load the data
#
# In this case we will explicitly set the background, RMF, and ARF in XSPEC
# If all files are linked correctly, this is not necessary as XSPEC loads
# the required files automatically

xspec.AllData.clear() # <= just to be sure there is nothing in the way

olddir = os.getcwd()
os.chdir('example_data/athena/')

sp = xspec.Spectrum('example-file.fak')

os.chdir(olddir)

# NOTE: in our example case, the response file is linked directly
#       from the spetral file. It is, of course, also possible to
#       link these files manually. In PyXspec backround, RMF and 
#       ARF are all atributes of the 'Spectrum' class instance.
# 
# For example (setting background, RMF, and ARF explicitly):
#
#sp.background   = 'path/to/bg_file.bak'
#sp.response     = 'path/to/response_file.rmf'
#sp.response.arf = 'path/to/auxiliary_file.arf'

In [ ]:
# The loading above will have caused a few warning messages, but we can check whether
# everything is now in place
xspec.AllData.show()

In [ ]:
# Set the appropriate spectral range
sp.notice('all')
xspec.AllData.ignore('bad')
sp.ignore('**-1.0 10.0-**')

In [ ]:
# A final look to make sure everything is in place
xspec.AllData.show()

#################################################################
#### NOTE:
From this point on we will use Python methods  to visualise the data, as implemented in PlotXspec. However, XSpec's normal visualisation can of course also still be used instead. For example, to make use of one of XSpec's plotting windows, enter

```python
xspec.Plot.device = "/xs"
```
and then pass any of XSpec's plotting arguments as strings to xspec.Plot, e.g.
```python
xspec.Plot('ldata')
```
#################################################################

In [ ]:
# A first look at the data
px.first_look(ymin=-0.05,ymax=2.5,ylog=False, #<= several keyword arguments can be used, 
              rebinsig=5,rebinbnum=40)        #   to manipulate the plot

### B) Define and fit a model

We will now define a simple model and fit it to the data. For the model, we will use redshifted blackbody, modified by Galactic absorption.

There are many ways to set the parameters in XSpec, and we show only the simplest example here. For a slightly more complex usage, please see section 3) of this notebook.

In [ ]:
# The simplest way to define a model in PyXspec
#
# We define the model components and then take a look
# at the parameters we will need to set

xspec.AllModels.clear() # <= just to sure, remove any possible existing definitions
mod = xspec.Model("wabs*pow+gauss") # <= the same model as used to create the spectrum

In [ ]:
# Now we set each of the parameters to an initial value.
# We can access the models and the parameters by name:
mod.wabs.nH = 1e-2
mod.powerlaw.PhoIndex = 1.7
mod.powerlaw.norm = 1e-4
mod.gaussian.LineE = 6.4
mod.gaussian.Sigma = 0.1
mod.gaussian.norm = 2

# If we want to freeze one of the parameters using 
# this syntax, we could specify, e.g.:
#
# mod.gaussian.LineE.frozen = True

In [ ]:
# Let us have another look at the model, to make sure
# all parameters are set
xspec.AllModels.show()

In [ ]:
# Now we can run the fit

# some fitting meta-parameters
xspec.Fit.statMethod = 'chi'
xspec.Fit.nIterations = 1000
# and run...
xspec.Fit.perform()

In [ ]:
# With the fit completed, we can access the usual functionalities in XSpec
xspec.AllModels.calcFlux("1.0,10.0")

In [ ]:
# Should we want to look at the scaling factor from rate to flux, we could do the following
print(f'scaling rate (1-10keV) = {sp.flux[0]/sp.rate[0]:.3e}')

In [ ]:
# And we can look at the resulting fit
px.plot_model_and_data(ymin=1e-4,rebinsig=5,rebinbnum=40)

In [ ]:
# If we, e.g., would like to look at the unfolded spectrum,
# we can specify the 'plottype' keyword argument.
px.plot_model_and_data(ymin=9e-7,rebinsig=5,rebinbnum=40,plottype='unfolded')

##################

PlotXspec also includes two methods to print a quick overview of the fitting results to screen: print_model_results() & print_errors(). The print_errors method requires the user to specify the $\chi^2$ level for which the errors should be calculated. Of course, if the overall $\chi^2$ statistic for the fit is too high, no error can be calculated (as XSpec raises an exception).

In [ ]:
px.print_model_results()

In [ ]:
px.print_errors(1.0)

##  2)  Further Plotting Examples

In some cases, it can be usefule to explore the parameter space of a model, to get a better understanding of the distribution of $\chi^2$, and therefore of the uncertainties. We can do this with steppar. The command for steppar is passed as a string and is the same as it would be in XSpec (we can see what the indices are for the model parameters of interest using e.g. the output of print_model_results or xspec.AllModels.show() ).

In [ ]:
# when running in Jupyer notebooks, it can be helpful to reduce the log-output here (especially for steppar)
# -- BXA's 'XSilence' method is helpful for this too --
xchat = xspec.Xset.chatter, xspec.Xset.logChatter
xspec.Xset.chatter    = 0
xspec.Xset.logChatter = 0

In [ ]:
# run steppar and plot. Here we vary parameters 1 (nH) and 2 (Gamma)
xspec.Fit.steppar('1 7.5 15.5 50 2 1.15 2.9 50')
px.plot_chisq_contours()

If it is only a single parameter we are interested in, we should consider the 1-dimensional $\chi^2$ distribution. By passing the best-fit value of a particular parameter (in the example below: kT), we can get an estimate on the error in this way.

In [ ]:
par = mod.powerlaw.PhoIndex   #<= the parameter of interest
xspec.Fit.steppar('2 1.2 2.9 1000')
px.calc_error_from_1Dchisq(par,level=1)

In [ ]:
# set XSpec's outbut levels back to their original values
xspec.Xset.chatter, xspec.Xset.logChatter = xchat

In [ ]:
# Note that the error value is above is different from the initial estimate shown in the
# fitting results overview (see output below). This is because the method used above 
# provided a different (more refined) approach for the calculation of the uncertainties
xspec.AllModels.show()

## 3) More detailed fitting example (XMM-Newton: pn, MOS1, and MOS2)

When fitting multiple datasets simultaneously, we need to slightly adjust our fitting procedure. In the following we will load XMM-Newton data from all three EPIC instruments. To account for a possible offset in the flux level between the spectra, we will multiply our model with a constant factor, which will be allowed to vary among the spectra. All other model parameters will remain tied.

The example data used here are for an AGN showing intrinsic obscuration and a weak Iron K$\alpha$ line. We will fit these data with a redshifted power-law model, modified with both intrinsic and Galactic absorption.

In [ ]:
# Load the data
xspec.AllData.clear()
xspec.AllModels.clear()

olddir = os.getcwd()
os.chdir('example_data/xmm/')

epicfn = 'epic_pn_agn.fak'
mos1fn = 'epic_mos1_agn.fak'
mos2fn = 'epic_mos2_agn.fak'

# load into different data groups
xspec.AllData(f"1:1 {epicfn} 2:2 {mos1fn} 3:3 {mos2fn}")

os.chdir(olddir)

# Spectral range
xspec.AllData.notice('all')
xspec.AllData.ignore('bad')
for ii in range(1,4):
    xspec.AllData(ii).ignore('**-0.2 10.0-**')

In [ ]:
# A first look at the data
px.first_look(ymin=1e-4,ymax=1e-1,rebinsig=5,rebinbnum=20)

In [ ]:
# Set all necessary parameters in XSpec & define the model
#
# In the simulated data, there is an offset in flux between the pn and the
# MOS1 & MOS2 spectra. We will account for this using multiplicative
# constant in the model. In the model definition, we need to make sure only
# the 'constant' factor is allowed to vary for the MOS1 & MOS2 spectra, 
# w.r.t. the pn spectrum. All other parameters should remain tied together.

## initial settings
xspec.Xset.abund = 'wilm' # Wilms et al. '00'
xspec.Xset.xsect = 'vern' # Verner et al. '96'
xspec.Xset.cosmo = '70 0 0.73'

z_opt = 0.015 # assume this is known

xspec.Fit.statMethod = 'chi'

## define model
xspec.AllModels += ("constant(zTBabs*(zpowerlw))")
mod = xspec.AllModels(1)  # <= we explicitly set mod to the first of the models 
                          #   (XSspec has now loaded three, one for each spectrum)

# Let us have a first look
# xspec.AllModels.show() # <= only necessary if the model is not automatically printed below

In [ ]:
# Silence XSpec for the moment...
xchat = xspec.Xset.chatter, xspec.Xset.logChatter
xspec.Xset.chatter    = 0
xspec.Xset.logChatter = 0

# Set parameter definitions
mod.constant.factor.values   = (1,-1)                           #<= note the different notation here; 
                                                                #   1 is the initial value, -1 means frozen
mod.zTBabs.Redshift.values   = (z_opt, -1.)
mod.zTBabs.nH.values         = (1., 0.01, 1e-3, 1e-3, 10., 10.) #<= standard XSpec syntax for parameter definition

mod.zpowerlw.PhoIndex.values = (1.8, 0.1, -0.5, -0.5, 5., 5.)
mod.zpowerlw.Redshift.values = (z_opt, -1)
mod.zpowerlw.norm.values     = (1.e-1, 0.01, 1.e-5, 1.e-5, 1e1, 1e1)

# Below we untie the model parameters for the MOS data. As noted,
# there are in fact three models (one per data group). In PyXspec
# we can access each model by its index in the AllModels object.
# Below, we first untie the contant parameter from its counterpart
# in the first model and then unfreeze it, by giving it a possible
# range of values.
xspec.AllModels(2).constant.factor.untie()
xspec.AllModels(2).constant.factor.values = (1, 0.01, 0.5, 0.5, 1.5, 1.5)
xspec.AllModels(3).constant.factor.untie()
xspec.AllModels(3).constant.factor.values = (1, 0.01, 0.5, 0.5, 1.5, 1.5)

# ... and show XSpec output again
xspec.Xset.chatter, xspec.Xset.logChatter = xchat

# Let's have another look at the models now
xspec.AllModels.show()

In [ ]:
# now we can run the fit
xspec.Fit.perform()

In [ ]:
# ... and plot the results
px.plot_model_and_data(ymin=1e-4,ymax=1,rebinsig=5,rebinbnum=40)

In [ ]:
# or if we want to look at e.g. only the MOS1 spectrum
px.plot_model_and_data(ymin=1e-4,ymax=1,idsp=2,rebinsig=5)

In [ ]:
# A quick overview of the fit results
px.print_model_results()

In [ ]:
# ... and of the errors
px.print_errors(1.0)